In [ ]:
import torch
import esm
import collections
import pandas as pd
import numpy as np

def esm_embeddings(peptide_sequence_list, flag):
    model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
    batch_converter = alphabet.get_batch_converter()
    model.eval()
    if flag == 1:
        print(model)
    labels, strs, tokens = batch_converter(peptide_sequence_list)
    lens = (tokens != alphabet.padding_idx).sum(1)
    with torch.no_grad():
        results = model(tokens, repr_layers=[6], return_contacts=True)
    token_representations = results["representations"][6]
    sequence_representations = []
    for i, tokens_len in enumerate(lens):
        sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
    embeddings = collections.defaultdict(list)
    for i in range(len(sequence_representations)):
        seq_rep = sequence_representations[i].tolist()
        for rep in seq_rep:
            embeddings[i].append(rep)
    embeddings = pd.DataFrame(embeddings).T
    return embeddings

In [ ]:
dataset = pd.read_csv('./train.csv',na_filter = False)
# dataset = pd.read_csv('./test.csv',na_filter = False)
sequence = dataset['Sequence']
results = pd.DataFrame()
flag = 1
for seq in sequence:
    format_seq = [seq,seq]
    tuple_sequence = tuple(format_seq)
    sequence_list = []
    sequence_list.append(tuple_sequence)
    seq_embedding = esm_embeddings(sequence_list, flag)
    flag += 1
    results = pd.concat([results,seq_embedding])

results.to_csv('./sequence_train_dense_output.csv')
# results.to_csv('./sequence_test_dense_output.csv')